In [10]:
from pyspark.sql import SparkSession

In [11]:
spark = SparkSession.builder \
    .appName("HELK JOIN") \
    .master("spark://helk-spark-master:7077") \
    .enableHiveSupport() \
    .getOrCreate()

In [12]:
es_reader = (spark.read
    .format("org.elasticsearch.spark.sql")
    .option("inferSchema", "true")
    .option("es.read.field.as.array.include", "tags")
    .option("es.nodes","helk-elasticsearch:9200")
    .option("es.net.http.auth.user","elastic")
)

In [13]:
security_df = es_reader.load("logs-endpoint-winevent-security-*/")

In [14]:
security_df.createOrReplaceTempView("security_events")

In [15]:
sysmon_df = es_reader.load("logs-endpoint-winevent-sysmon-*/")

In [16]:
sysmon_df.createOrReplaceTempView("sysmon_events")

In [28]:
process_hollowing = spark.sql(
'''
SELECT 
    c.host_name, 
    c.user_name,
    c.process_parent_name,
    d.process_target_name,
    c.user_logon_id AS Sysmon_EID_1_LogonId,
    d.process_granted_access
FROM security_events b
JOIN sysmon_events c
    ON c.process_name = b.process_name
    AND c.event_id = 1
    AND c.user_name = b.user_name
JOIN sysmon_events d
    ON c.process_guid = d.process_target_guid
    AND d.event_id = 10
    AND (d.process_granted_access & 2312) == 2312 -- 2312 = (0x908) process access rights for  PROCESS_VM_WRITE & PROCESS_VM_OPERATION & PROCESS_SUSPEND_RESUME & PROCESS_CREATE_PROCESS
    AND c.process_parent_guid = d.process_guid
    AND c.process_name = d.process_target_name
JOIN sysmon_events e
    ON e.process_guid = c.process_parent_guid
    AND e.event_id = 5
    AND e.process_name = d.process_name
    AND c.process_parent_name = e.process_name 
JOIN sysmon_events f
    ON f.process_guid = c.process_parent_guid
    AND f.event_id = 7
    AND f.process_path = c.process_parent_path
    AND f.process_path = f.module_loaded
    AND f.signed = "false"
WHERE 
    b.event_id = 4688
'''
).show(10,False)

+------------------+---------+--------------------+-------------------+--------------------+----------------------+
|host_name         |user_name|process_parent_name |process_target_name|Sysmon_EID_1_LogonId|process_granted_access|
+------------------+---------+--------------------+-------------------+--------------------+----------------------+
|win10.marvel.local|thor     |processhollowing.exe|cmd.exe            |9340634             |2097151               |
|win10.marvel.local|thor     |processhollowing.exe|svchost.exe        |9340634             |2097151               |
+------------------+---------+--------------------+-------------------+--------------------+----------------------+

